<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [1]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings

warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [2]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
# os.environ["HOPSWORKS_API_KEY"] = ""

project = hopsworks.login()
fs = project.get_feature_store()
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1149079
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "france", "city": "paris", "street": "18eme", "aqicn_url": "https://api.waqi.info/feed/@3082", "latitude": 48.86, "longitude": 2.32}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [3]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [4]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,1.0,france,paris,18eme,2024-11-20,https://api.waqi.info/feed/@3082


In [5]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 172.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [6]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 48.75°N 2.25°E
Elevation 46.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-20,6.850000,0.0,16.179985,290.854523,paris
1,2024-11-21,1.650000,0.8,13.493999,80.789047,paris
2,2024-11-22,3.600000,0.0,17.414476,262.875092,paris
3,2024-11-23,4.800000,0.0,20.570463,175.985901,paris
4,2024-11-24,17.450001,0.0,27.609911,202.217651,paris
5,2024-11-25,12.800000,0.6,17.388409,242.904526,paris
6,2024-11-26,11.150000,0.0,21.674870,237.893768,paris
7,2024-11-27,8.500000,0.0,5.506941,191.309891,paris
8,2024-11-28,7.950000,0.0,9.339208,152.447281,paris
9,2024-11-29,14.200000,0.0,21.203358,220.179169,paris


In [7]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 448.0+ bytes


In [8]:
df_aq = air_quality_fg.read(read_options={"use_hive": True})
df_aq.head()

Finished: Reading data from Hopsworks, using Hive (2.81s) 


,date,pm25,country,city,street,url,pm25_3days
0,2024-01-20,66.0,france,paris,18eme,https://api.waqi.info/feed/@3082,67.333333
1,2022-06-19,57.0,france,paris,18eme,https://api.waqi.info/feed/@3082,66.000000
2,2024-04-19,23.0,france,paris,18eme,https://api.waqi.info/feed/@3082,61.500000
3,2024-08-13,38.0,france,paris,18eme,https://api.waqi.info/feed/@3082,48.666667
4,2022-11-25,51.0,france,paris,18eme,https://api.waqi.info/feed/@3082,39.333333


In [9]:
df_aq = df_aq._append(aq_today_df)
df_aq['pm25_3days'] = df_aq['pm25'].shift(1).rolling(window=3, min_periods=1).mean()
df_aq = df_aq.dropna(subset=['pm25_3days'])
df_aq.head()

,date,pm25,country,city,street,url,pm25_3days
1,2022-06-19,57.0,france,paris,18eme,https://api.waqi.info/feed/@3082,66.000000
2,2024-04-19,23.0,france,paris,18eme,https://api.waqi.info/feed/@3082,61.500000
3,2024-08-13,38.0,france,paris,18eme,https://api.waqi.info/feed/@3082,48.666667
4,2022-11-25,51.0,france,paris,18eme,https://api.waqi.info/feed/@3082,39.333333
5,2021-12-23,87.0,france,paris,18eme,https://api.waqi.info/feed/@3082,37.333333


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [10]:
# Insert new data
air_quality_fg.insert(df_aq)

FeatureStoreException: Features are not compatible with Feature Group schema: 
 - pm25 (expected type: 'float', derived from input: 'double') has the wrong type.
Note that feature (or column) names are case insensitive and spaces are automatically replaced with underscores.

In [ ]:
# Insert new data
weather_fg.insert(daily_df)

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
